In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import cv2
from PIL import Image

In [2]:
input_path = "../data/archive/raw-img/"

height = 299
width = 299
channels = 3

# Number of total classes
total_classes = 10
classes = ['butterfly',
 'cat',
 'chicken',
 'cow',
 'dog',
 'elephant',
 'horse',
 'sheep',
 'spider',
 'squirrel']

# Dimensions of our images
height = 299
width = 299
channels = 3

image_data = []
image_labels = []

In [6]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255,
                                                           validation_split=0.2
                                                       )

In [ ]:
# Load the images from the correct path
for i in classes:
    path = input_path + i
    images = os.listdir(path)
    count = 0
    
    for img in images:
        if count%200 == 0:
            print(f"Count {count} for class {i}")
        try:
            image = cv2.imread(path + '/' + img)
            image_fromarray = Image.fromarray(image, "RGB")
            resize_image = image_fromarray.resize((height, width))
            image_data.append(np.array(resize_image))
            image_labels.append(i)
            count += 1
        except:
            print("Error in Image loading")
    print(f"Class {i} completed")
            
# Converting lists into numpy arrays
image_data = np.array(image_data)
image_labels = np.array(image_labels)

# Time taken to load our images in seconds
end = time.time()
print("Time taken: ", round(end-start, 5), "seconds")

Count 0 for class butterfly
Count 200 for class butterfly
Count 400 for class butterfly
Count 600 for class butterfly
Count 800 for class butterfly
Count 1000 for class butterfly
Count 1200 for class butterfly
Count 1400 for class butterfly
Count 1600 for class butterfly
Count 1800 for class butterfly
Count 2000 for class butterfly
Class butterfly completed
Count 0 for class cat
Count 200 for class cat
Count 400 for class cat
Count 600 for class cat
Count 800 for class cat
Count 1000 for class cat
Count 1200 for class cat
Count 1400 for class cat
Count 1600 for class cat
Class cat completed
Count 0 for class chicken
Count 200 for class chicken
Count 400 for class chicken
Count 600 for class chicken
Count 800 for class chicken
Count 1000 for class chicken
Count 1200 for class chicken
Count 1400 for class chicken
Count 1600 for class chicken
Count 1800 for class chicken
Count 2000 for class chicken
Count 2200 for class chicken
Count 2400 for class chicken
Count 2600 for class chicken
Cou

In [44]:
train_datagen = train_gen.flow(input_path,
                              target_size=(height, width),
                              color_mode='rgb',
                              batch_size=32,
                              shuffle=True,
                              subset='training')
val_datagen = train_gen.flow(input_path,
                              target_size=(height, width),
                              color_mode='rgb',
                              batch_size=32,
                              shuffle=True,
                              subset='validation')

Found 20947 images belonging to 10 classes.
Found 5232 images belonging to 10 classes.


In [45]:
train_datagen.class_indices

{'butterfly': 0,
 'cat': 1,
 'chicken': 2,
 'cow': 3,
 'dog': 4,
 'elephant': 5,
 'horse': 6,
 'sheep': 7,
 'spider': 8,
 'squirrel': 9}

In [61]:
new_input = tf.keras.layers.Input(shape=(299, 299, 3))

In [77]:
tl_model = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_tensor=new_input)    

In [78]:
tl_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 299, 299, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 299, 299, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 299, 299, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 149, 149, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 149, 149, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 149, 149, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 74, 74, 128)       0     

In [79]:
Layers = tl_model.layers
# print(Layers[-2].name)
for layers in Layers:
    layers.trainable = False

In [80]:
print(f"Total layers : {len(Layers)}")

Total layers : 19


In [81]:
model = tf.keras.Sequential()
model.add(tl_model)
model.add(tf.keras.layers.Flatten())
#add dense layer
model.add(tf.keras.layers.Dense(36,activation='relu'))

# #add droput to avoid overfitting
# model.add(tf.keras.layers.Dropout(0.5))

# #BatchNormaliztion
# model.add(tf.keras.layers.BatchNormalization())

#add softmax layer with 12 output labels
model.add(tf.keras.layers.Dense(10,activation='softmax'))

In [82]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 9, 9, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 41472)             0         
_________________________________________________________________
batch_normalization_286 (Bat (None, 41472)             165888    
_________________________________________________________________
dense_5 (Dense)              (None, 256)               10617088  
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
batch_normalization_287 (Bat (None, 256)               1024      
_________________________________________________________________
dense_6 (Dense)              (None, 128)              

In [83]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

/home/smit/.local/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [84]:
model.fit_generator(train_datagen, epochs=10, validation_data=val_datagen)

/home/smit/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
  3/655 [..............................] - ETA: 2:55:47 - loss: 3.0664 - accuracy: 0.1042

KeyboardInterrupt: 

In [63]:
tl_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 149, 149, 32) 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 149, 149, 32) 0           batch_normalization_94[0][0]     
_______________________________________________________________________________________

In [41]:
base_input = tl_model.layers[0].input
base_output = tl_model.layers[-2].output
final_output = tf.keras.layers.Dense(10, activation = 'softmax')(base_output)
new_model = tf.keras.Model(inputs = base_input, outputs = final_output)

In [46]:
new_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_121 (Conv2D)             (None, 149, 149, 32) 864         input_8[0][0]                    
__________________________________________________________________________________________________
batch_normalization_120 (BatchN (None, 149, 149, 32) 96          conv2d_121[0][0]                 
__________________________________________________________________________________________________
activation_120 (Activation)     (None, 149, 149, 32) 0           batch_normalization_120[0][0]    
____________________________________________________________________________________________

activation_180 (Activation)     (None, 17, 17, 192)  0           batch_normalization_180[0][0]    
__________________________________________________________________________________________________
activation_183 (Activation)     (None, 17, 17, 192)  0           batch_normalization_183[0][0]    
__________________________________________________________________________________________________
activation_188 (Activation)     (None, 17, 17, 192)  0           batch_normalization_188[0][0]    
__________________________________________________________________________________________________
activation_189 (Activation)     (None, 17, 17, 192)  0           batch_normalization_189[0][0]    
__________________________________________________________________________________________________
mixed7 (Concatenate)            (None, 17, 17, 768)  0           activation_180[0][0]             
                                                                 activation_183[0][0]             
          

In [49]:
new_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics=['accuracy'])

In [51]:
tf.keras.backend.clear_session()

In [56]:
Layers = new_model.layers
# print(Layers[-2].name)
for layers in Layers:
    layers.trainable = False

In [57]:
new_model.fit_generator(train_datagen,
                       epochs=10,
                       verbose=1,
                       validation_data=val_datagen)

/home/smit/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
  2/655 [..............................] - ETA: 2:25:58 - loss: 1.1801 - accuracy: 0.5938

KeyboardInterrupt: 

In [ ]:
tl_model.layers